<a href="https://colab.research.google.com/github/RhysWangJunfei/nilm/blob/master/LH/lh_ac_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import tensorflow as tf
import numpy as np
import io
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from google.colab import files
import matplotlib.pyplot as plt

In [0]:
'''Sliding window function'''
def create_dataset(dataset, look_back=1):
    dataX = []
    for i in range(len(dataset)-look_back+1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
    return np.array(dataX)

In [0]:
uploaded = files.upload()
'''Load data'''
ac_data = pd.read_csv(io.BytesIO(uploaded['lh_ac.csv']))
#ac_data = pd.read_csv('lh_ac.csv')

Saving lh_ac.csv to lh_ac.csv


In [0]:
window_size=60
dataX_raw = create_dataset(ac_data['meter_power'].as_matrix(), window_size)

In [0]:
dataX_raw.shape

(426081, 60)

In [0]:

Y_raw = ac_data['device_power'].values[window_size-1:,].reshape([-1,1])
Y_raw.shape
categorized_Y = np.ones(Y_raw.shape)*3
categorized_Y[[np.where(Y_raw==0)[0]],:]=0
categorized_Y[[np.where((Y_raw>0)&(Y_raw<=100))[0]],:]=1
categorized_Y[[np.where((Y_raw>100)&(Y_raw<=1000))[0]],:]=2
categorized_Y[np.where((Y_raw>1000))[0],:]=3

encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
Y_1hot = encoder.fit_transform(categorized_Y)

X_train, X_test, y_train, y_test = train_test_split(dataX_raw, Y_1hot, test_size=0.1, shuffle=True)
scaler = MinMaxScaler()
scaler.fit(X_train.astype(float))
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [0]:
df = pd.Series(categorized_Y.flatten())
df.value_counts()

0.0    242622
1.0    153865
3.0     20356
2.0      9238
dtype: int64

In [0]:
from imblearn.under_sampling import RandomUnderSampler
tl = RandomUnderSampler(sampling_strategy={0:8000,1:8000,2:8000,3:8000})
X_train, y_train = tl.fit_resample(X_train, y_train)
whole_train = np.concatenate([X_train,y_train],axis=1)

In [0]:
'''Hyper parameters for deep learning'''
# Hyper Parameters
LR = 0.0001               # learning rate
#cfg_list = nf.model_configs()
#error_list = []

#hyperparameters
batch_size=256
unit_num=128

In [0]:
'''RNN Model Definition'''
tf.reset_default_graph()
''''''
#define inputs
tf_x = tf.placeholder(tf.float32, [None, window_size,1],name='x')
tf_y = tf.placeholder(tf.int32, [None, 4],name='y')

lstm_cell =tf.contrib.rnn.BasicLSTMCell(num_units=unit_num,name='lstm_cell')
outputs, (h_c, h_n) = tf.nn.dynamic_rnn(
    lstm_cell,                   # cell you have chosen
    tf_x,                      # input
    initial_state=None,         # the initial hidden state
    dtype=tf.float32,           # must given if set initial_state = None
    time_major=False,           # False: (batch, time step, input); True: (time step, batch, input)
)
l1 = tf.layers.dense(outputs[:, -1, :],512,activation=tf.nn.leaky_relu,name='l1')
l2 = tf.layers.dense(l1,1024,activation=tf.nn.leaky_relu,name='l2')
l3 = tf.layers.dense(l2,512,activation=tf.nn.leaky_relu,name='l3')
l4 = tf.layers.dense(l3,256,activation=tf.nn.leaky_relu,name='l4')
l5 = tf.layers.dense(l4,128,activation=tf.nn.leaky_relu,name='l5')
l6 = tf.layers.dense(l5,84,activation=tf.nn.leaky_relu,name='l6')
l7 = tf.layers.dense(l6,64,activation=tf.nn.leaky_relu,name='l7')
l8 = tf.layers.dense(l7,48,activation=tf.nn.leaky_relu,name='l8')
l9 = tf.layers.dense(l8,32,activation=tf.nn.leaky_relu,name='l9')
l10 = tf.layers.dense(l9,24,activation=tf.nn.leaky_relu,name='l10')
l11 = tf.layers.dense(l10,16,activation=tf.nn.leaky_relu,name='l11')
l12 = tf.layers.dense(l11,8,activation=tf.nn.leaky_relu,name='l12')
pred = tf.layers.dense(l12,4,activation=tf.nn.relu,name='pred')

with tf.name_scope('loss'):
    cross_entropy =  tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_y, logits=pred) 
    loss = tf.reduce_mean(cross_entropy)
    tf.summary.scalar("loss",tensor=loss)

train_op = tf.train.AdamOptimizer(LR).minimize(loss)

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tf_y, axis=1), tf.argmax(pred, axis=1)), tf.float32))

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()) 
saver = tf.train.Saver()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.


In [0]:
from imblearn.under_sampling import RandomUnderSampler
tl = RandomUnderSampler(sampling_strategy={0:8000,1:8000,2:8000,3:8000})

sess = tf.Session()
#sess.run(init_op)
saver.restore(sess, 'my_net/save_net_rnn.ckpt')

for i in range(0,10):
  X_train, y_train = tl.fit_resample(X_train, y_train)
  whole_train = np.concatenate([X_train,y_train],axis=1)
  for j in range(0,100):
    print('Loop'+str(i)+',###iteration: '+str(j)+'###')
    batch_index = np.random.choice(32000,batch_size)
    batch_train = whole_train[batch_index,:]
    batch_X = batch_train[:,:-4].reshape([batch_size,window_size,1])
    batch_y = batch_train[:,-4:]
    sess.run(train_op,{tf_x:batch_X , tf_y:batch_y})
    cost_ = sess.run(loss,{tf_x:batch_X, tf_y:batch_y})
    acc_train = sess.run(accuracy,{tf_x:batch_X, tf_y:batch_y})
    acc_test = sess.run(accuracy,feed_dict={tf_x: X_test.reshape([X_test.shape[0],window_size,1]), tf_y:y_test})
    print('train loss= %.4f' % cost_+', Acc=%.2f'% acc_train)
    print('Test Acc=%.2f'% acc_test)
    
    pre = sess.run(pred,feed_dict={tf_x: batch_X, tf_y:batch_y})
    y_lables_argmax = tf.argmax(tf_y,axis=1)  
    y_pred_argmax = tf.argmax(pre,axis=1)
    confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=4)
    #print('Confusion Matrix: \n\n', tf.Tensor.eval(confusion,feed_dict=None))
    print(confusion.eval(session=sess,feed_dict={tf_x: batch_X, tf_y:batch_y}))
    if(j//99==0):
      save_path = saver.save(sess, "my_net/save_net_rnn.ckpt")
pre = sess.run(pred,feed_dict={tf_x: X_test.reshape([X_test.shape[0],window_size,1]), tf_y: y_test})
y_lables_argmax = np.argmax(y_test,1)
y_pred_argmax = np.argmax(pre,1)
confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=4)
#print('Confusion Matrix: \n\n', tf.Tensor.eval(confusion,feed_dict=None))
print(confusion.eval(session=sess))
sess.close()


INFO:tensorflow:Restoring parameters from my_net/save_net_rnn.ckpt
Loop0,###iteration: 0###
train loss= 0.5034, Acc=0.80
Test Acc=0.90
[[52  3  0  5]
 [ 1 66  0  0]
 [20  9 43  5]
 [ 4  0  3 45]]
Loop0,###iteration: 1###
train loss= 0.4170, Acc=0.86
Test Acc=0.90
[[57  1  2  4]
 [ 1 62  0  0]
 [12  7 38  4]
 [ 4  0  2 62]]
Loop0,###iteration: 2###
train loss= 0.5513, Acc=0.80
Test Acc=0.89
[[59  3  3  6]
 [ 2 55  2  0]
 [16  8 31  3]
 [ 5  0  2 61]]
Loop0,###iteration: 3###
train loss= 0.5426, Acc=0.81
Test Acc=0.89
[[57  0  3  3]
 [ 2 52  4  0]
 [13 11 43  6]
 [ 3  0  3 56]]
Loop0,###iteration: 4###
train loss= 0.4673, Acc=0.83
Test Acc=0.89
[[54  2  0  6]
 [ 5 54  0  0]
 [14 12 35  3]
 [ 2  0  0 69]]
Loop0,###iteration: 5###
train loss= 0.4131, Acc=0.86
Test Acc=0.89
[[64  1  1  4]
 [ 1 63  1  0]
 [11 10 41  4]
 [ 3  0  1 51]]
Loop0,###iteration: 6###
train loss= 0.5181, Acc=0.81
Test Acc=0.90
[[64  4  0  2]
 [ 4 58  0  0]
 [21  8 26  4]
 [ 3  0  2 60]]
Loop0,###iteration: 7###
train

In [0]:
!mkdir my_net_ac

In [0]:
mv checkpoint save_net_rnn.ckpt.data-00000-of-00001 save_net_rnn.ckpt.index save_net_rnn.ckpt.meta my_net_ac

In [0]:
mv my_net_ac my_net